#### Load dataset (student protugese scores)

In [1]:
import pandas as pd
data = pd.read_csv('student-por.csv',sep=';')
data

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,0,11,13,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,MS,F,19,R,GT3,T,2,3,services,other,...,5,4,2,1,2,5,4,10,11,10
645,MS,F,18,U,LE3,T,3,1,teacher,services,...,4,3,4,1,1,1,4,15,15,16
646,MS,F,18,U,GT3,T,1,1,other,other,...,1,1,1,1,1,5,6,11,12,9
647,MS,M,17,U,LE3,T,3,1,services,services,...,2,4,5,3,4,2,6,10,10,10


In [4]:
# generate binary label for (pass/fail) based on G1+g2+g3( test-grade, each 0-20pts)threshold for passing is sum=>35
data['pass']= data.apply(lambda row: 1 if(row['G1']+row['G2']+row['G3'])>= 35 else 0, axis=1)
data = data.drop(['G1','G2','G3'], axis=1)
data.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,pass
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,no,no,4,3,4,1,1,3,4,0
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,no,5,3,3,1,1,3,2,0
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,no,4,3,2,2,3,3,6,1
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,yes,3,2,2,1,1,5,0,1
4,GP,F,16,U,GT3,T,3,3,other,other,...,no,no,4,3,2,1,2,5,0,1


In [5]:
# ONE-HOT ENCODING  ausin get_dummies feature
data = pd.get_dummies(data, columns=['sex','school','address','famsize','Pstatus','Mjob','Fjob','reason',
                            'guardian','schoolsup','famsup','paid','activities',
                                     'nursery','higher','internet','romantic'])
data.head()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,...,activities_no,activities_yes,nursery_no,nursery_yes,higher_no,higher_yes,internet_no,internet_yes,romantic_no,romantic_yes
0,18,4,4,2,2,0,4,3,4,1,...,1,0,0,1,0,1,1,0,1,0
1,17,1,1,1,2,0,5,3,3,1,...,1,0,1,0,0,1,0,1,1,0
2,15,1,1,1,2,0,4,3,2,2,...,1,0,0,1,0,1,0,1,1,0
3,15,4,2,1,3,0,3,2,2,1,...,0,1,0,1,0,1,0,1,0,1
4,16,3,3,1,2,0,4,3,2,1,...,1,0,0,1,0,1,1,0,1,0


In [10]:
# shuffle data
data = data.sample(frac=1)

#train set and test set
d_train = data[:500]
d_test = data[500:]

# train target pass and fail df
d_train_fail = d_train.drop(['pass'], axis=1)
d_train_pass = d_train['pass']

# test target pass and fail df
d_test_fail = d_test.drop(['pass'], axis=1)
d_test_pass = d_test['pass']

# data target pass and fail df
data_fail = data.drop(['pass'], axis=1)
data_pass = data['pass']

import numpy as np
print(f'Passing: {np.sum(data_pass)} out of {len(data)} {round(100*float(np.sum(data_pass))/len(data_pass),2)}')
                  

Passing: 328 out of 649 50.54


In [16]:
# fit a decision tree using scikit-learn
from sklearn import tree
t = tree.DecisionTreeClassifier(criterion='entropy', max_depth=5)
t = t.fit(d_train_fail, d_train_pass)

In [25]:
# save_tree
tree.export_graphviz(t, out_file=None, label="all",impurity=False,proportion=True,
                 feature_names=list(d_train_fail),class_names=['fail','pass'], filled=True,rounded=True)

'digraph Tree {\nnode [shape=box, style="filled, rounded", color="black", fontname="helvetica"] ;\nedge [fontname="helvetica"] ;\n0 [label="failures <= 0.5\\nsamples = 100.0%\\nvalue = [0.488, 0.512]\\nclass = pass", fillcolor="#f6fafe"] ;\n1 [label="higher_yes <= 0.5\\nsamples = 84.6%\\nvalue = [0.407, 0.593]\\nclass = pass", fillcolor="#c1e0f7"] ;\n0 -> 1 [labeldistance=2.5, labelangle=45, headlabel="True"] ;\n2 [label="famsup_yes <= 0.5\\nsamples = 5.6%\\nvalue = [0.929, 0.071]\\nclass = fail", fillcolor="#e78b48"] ;\n1 -> 2 ;\n3 [label="Medu <= 2.5\\nsamples = 2.6%\\nvalue = [0.846, 0.154]\\nclass = fail", fillcolor="#ea985d"] ;\n2 -> 3 ;\n4 [label="Dalc <= 2.5\\nsamples = 2.4%\\nvalue = [0.917, 0.083]\\nclass = fail", fillcolor="#e78c4b"] ;\n3 -> 4 ;\n5 [label="samples = 1.8%\\nvalue = [1.0, 0.0]\\nclass = fail", fillcolor="#e58139"] ;\n4 -> 5 ;\n6 [label="samples = 0.6%\\nvalue = [0.667, 0.333]\\nclass = fail", fillcolor="#f2c09c"] ;\n4 -> 6 ;\n7 [label="samples = 0.2%\\nvalue = 